In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from embfts.models.MimoNonStationaryFtsPca import MimoNonStationaryFtsPca
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math
from embfts.util.DataSetUtil import DataSetUtil


np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [3]:
data_set_util = DataSetUtil()

In [4]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/AirQualityUCI.csv', sep=';', decimal=',')
data = df.drop(labels=['Date','Time','Unnamed: 15', 'Unnamed: 16'], axis=1)
data.dropna(inplace=True)
#data = clean_dataset(data)
data.head()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
1,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888


In [5]:
data.describe()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
count,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000
mean,-34.207524,1048.990061,-159.090093,1.865683,894.595276,168.616971,794.990168,58.148873,1391.479641,975.072032,9.778305,39.485380,-6.837604
std,77.657170,329.832710,139.789093,41.380206,342.333252,257.433866,321.993552,126.940455,467.210125,456.938184,43.203623,51.216145,38.976670
min,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000
25%,0.600000,921.000000,-200.000000,4.000000,711.000000,50.000000,637.000000,53.000000,1185.000000,700.000000,10.900000,34.100000,0.692300
50%,1.500000,1053.000000,-200.000000,7.900000,895.000000,141.000000,794.000000,96.000000,1446.000000,942.000000,17.200000,48.600000,0.976800
75%,2.600000,1221.000000,-200.000000,13.600000,1105.000000,284.000000,960.000000,133.000000,1662.000000,1255.000000,24.100000,61.900000,1.296200
max,11.900000,2040.000000,1189.000000,63.700000,2214.000000,1479.000000,2683.000000,340.000000,2775.000000,2523.000000,44.600000,88.700000,2.231000


In [6]:
# frame as supervised learning
reframed = data_set_util.series_to_supervised_mimo(data, 1, 1)
reframed.head()

,CO(GT)(t-1),PT08.S1(CO)(t-1),NMHC(GT)(t-1),C6H6(GT)(t-1),PT08.S2(NMHC)(t-1),NOx(GT)(t-1),PT08.S3(NOx)(t-1),NO2(GT)(t-1),PT08.S4(NO2)(t-1),PT08.S5(O3)(t-1),...,C6H6(GT)(t),PT08.S2(NMHC)(t),NOx(GT)(t),PT08.S3(NOx)(t),NO2(GT)(t),PT08.S4(NO2)(t),PT08.S5(O3)(t),T(t),RH(t),AH(t)
1,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,...,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,...,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,...,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,...,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888
5,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,...,4.7,750.0,89.0,1337.0,96.0,1393.0,949.0,11.2,59.2,0.7848


In [7]:
cols = reframed.columns
cols

Index(['CO(GT)(t-1)', 'PT08.S1(CO)(t-1)', 'NMHC(GT)(t-1)', 'C6H6(GT)(t-1)',
       'PT08.S2(NMHC)(t-1)', 'NOx(GT)(t-1)', 'PT08.S3(NOx)(t-1)',
       'NO2(GT)(t-1)', 'PT08.S4(NO2)(t-1)', 'PT08.S5(O3)(t-1)', 'T(t-1)',
       'RH(t-1)', 'AH(t-1)', 'CO(GT)(t)', 'PT08.S1(CO)(t)', 'NMHC(GT)(t)',
       'C6H6(GT)(t)', 'PT08.S2(NMHC)(t)', 'NOx(GT)(t)', 'PT08.S3(NOx)(t)',
       'NO2(GT)(t)', 'PT08.S4(NO2)(t)', 'PT08.S5(O3)(t)', 'T(t)', 'RH(t)',
       'AH(t)'],
      dtype='object')

In [8]:
df_forecats_columns = ['CO(GT)(t)', 'PT08.S1(CO)(t)', 'NMHC(GT)(t)',
       'C6H6(GT)(t)', 'PT08.S2(NMHC)(t)', 'NOx(GT)(t)', 'PT08.S3(NOx)(t)',
       'NO2(GT)(t)', 'PT08.S4(NO2)(t)', 'PT08.S5(O3)(t)', 'T(t)', 'RH(t)',
       'AH(t)']

In [9]:
def sliding_window_pca(data,n_windows,train_size,steps_ahead,transformation):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }


    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            df_train = ttrain.loc[:,'CO(GT)(t-1)':'AH(t-1)']
            df_test = ttest.loc[:,'CO(GT)(t-1)':'AH(t-1)']
            df_original = ttest.loc[:,'CO(GT)(t)':'AH(t)']
            
            models, data_train = mimo_kpca_nsfts.run_train(df_train,transformation)
            forecast, data_test = mimo_kpca_nsfts.run_test(models,df_test,steps_ahead,transformation)
            
            columns = list(df_forecats_columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

## Sliding Window: MIMO KPCA-NSFTS

In [10]:
num_components_pca = 3
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 25
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [11]:
kpca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='KPCA')

--------------------
training window 0
--------------------
training window 311
--------------------
training window 622
--------------------
training window 933
--------------------
training window 1244
--------------------
training window 1555
--------------------
training window 1866
--------------------
training window 2177
--------------------
training window 2488
--------------------
training window 2799
--------------------
training window 3110
--------------------
training window 3421
--------------------
training window 3732
--------------------
training window 4043
--------------------
training window 4354
--------------------
training window 4665
--------------------
training window 4976
--------------------
training window 5287
--------------------
training window 5598
--------------------
training window 5909
--------------------
training window 6220
--------------------
training window 6531
--------------------
training window 6842
--------------------
training window 715

In [12]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": []
}

measures = kpca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 



In [14]:
print("Statistics MIMO KPCA-NSFTS(test): ")
final_measures_kpca

Statistics MIMO KPCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse
0,CO(GT)(t),37.845,11.604,884.138,-0.377,36.647,inf
1,PT08.S1(CO)(t),127.399,80.355,9.870,0.583,4.596,0.132
2,NMHC(GT)(t),19.316,11.940,4.800,0.967,2.475,NaN
3,C6H6(GT)(t),11.053,5.013,68.967,0.483,19.809,0.139
4,PT08.S2(NMHC)(t),153.908,103.538,13.444,0.612,6.372,0.132
5,NOx(GT)(t),108.947,69.275,50.152,0.400,18.744,inf
6,PT08.S3(NOx)(t),121.153,84.641,12.449,0.678,6.049,0.117
7,NO2(GT)(t),71.424,35.052,37.286,0.075,14.641,inf
8,PT08.S4(NO2)(t),172.066,107.113,10.695,0.539,4.833,0.129
9,PT08.S5(O3)(t),188.322,133.322,16.613,0.650,7.864,0.129


In [15]:
final_measures_kpca.to_csv (r'kpca_mo_ensfts_uci_air_quality_italy.csv', index = False, header=True)

## Sliding Window: MIMO PCA-NSFTS

In [20]:
num_components_pca = 6
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 25
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [21]:
pca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='PCA')

--------------------
training window 0
--------------------
training window 311
--------------------
training window 622
--------------------
training window 933
--------------------
training window 1244
--------------------
training window 1555
--------------------
training window 1866
--------------------
training window 2177
--------------------
training window 2488
--------------------
training window 2799
--------------------
training window 3110
--------------------
training window 3421
--------------------
training window 3732
--------------------
training window 4043
--------------------
training window 4354
--------------------
training window 4665
--------------------
training window 4976
--------------------
training window 5287
--------------------
training window 5598
--------------------
training window 5909
--------------------
training window 6220
--------------------
training window 6531
--------------------
training window 6842
--------------------
training window 715

In [22]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": []
}

measures = pca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 

print("Statistics MIMO PCA-NSFTS(test): ")
final_measures_pca

Statistics MIMO PCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse
0,CO(GT)(t),40.307,18.438,1193.940,-0.313,58.975,NaN
1,PT08.S1(CO)(t),135.282,87.742,11.428,0.533,4.863,0.141
2,NMHC(GT)(t),20.049,14.010,6.473,0.964,3.258,NaN
3,C6H6(GT)(t),11.642,5.530,63.056,0.455,21.524,0.144
4,PT08.S2(NMHC)(t),163.239,112.330,15.617,0.569,7.057,0.139
5,NOx(GT)(t),112.374,76.307,53.693,0.427,22.353,NaN
6,PT08.S3(NOx)(t),135.492,97.680,16.434,0.628,7.475,0.129
7,NO2(GT)(t),69.897,40.202,39.474,0.191,18.718,NaN
8,PT08.S4(NO2)(t),182.044,116.135,12.704,0.486,4.962,0.136
9,PT08.S5(O3)(t),196.959,142.928,19.624,0.624,9.210,0.134


In [23]:
final_measures_pca.to_csv (r'pca_mo_ensfts_uci_air_quality_italy.csv', index = False, header=True)